In [ ]:
%pip -q install google-genai
# Instalar Framework de agentes do Google
%pip install -q google-adk


# Configura a API Key do Google Gemini

import os
from google.colab import userdata
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"


# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response


# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


##########################################
# --- Agente 1: Buscador de Destinos --- #
##########################################
def agente_buscador(destino, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um agente de viagens da agência Lima's Tur, uma das maiores agências de viagem do Brasil.
        A sua tarefa é usar a ferramenta de busca do google (google_search)
        para buscar informações relevantes apenas sobre a cidade de destino informada abaixo. Você deve ignorar a cidade de origem.
        Informações tais como pontos turísticos, restaurantes, atrações, acomodações
        o que fazer na cidade de destino informada, melhor época do ano para visitar, etc.
        Foque em informações relevantes, com base na quantidade e entusiasmo das avaliações sobre ela.
        Se um ponto turístico, restaurante ou acomodação tiver poucas notícias ou avaliações entusiasmadas, é possível que não seja tão relevante assim
        e pode ser substituído por outros que tenham mais.
        Essas informações relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        É imprescindível a inclusão dos links para os websites dos pontos turísticos, restaurantes, atrações, acomodações, etc. caso estejam disponíveis.
        """,
        description="Agente que busca informações no Google sobre determinado destino",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Destino: {destino}\nData de hoje: {data_de_hoje}"

    destino_buscado = call_agent(buscador, entrada_do_agente_buscador)
    return destino_buscado

################################################
# --- Agente 2: Planejador da viagem --- #
################################################
def agente_planejador(cidade_destino, estado_destino, cidade_origem, estado_origem):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um planejador de viagens da agência Lima's Tur, uma das maiores agências de viagem do Brasil, especialista em
        encontrar passagens aéreas e terrestres mais baratas, acomodações, restaurantes, passeios, etc. para um destino específico.
        Com base na cidade de destino informada, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano de viagem o mais completo possível,
        com valores em reais (R$) de passagens aéres e terrestres, caso tenham. Você também pode usar o Google (google_search) para encontrar mais
        informações sobre cidade de destino e aprofundar. Esse plano de viagem deverá ser completo e detalhado e deverá levar em consideração a cidade de origem
        e o estado de origem, para traçar a rota e informar os valores de transporte. Informe os valores sempre em reais do Brasil (R$).
        Ao final, você irá escolher as melhores opções com base nas suas pesquisas
        e retornar esse plano de viagem completo, com seus pontos mais relevantes.
        Ao informar as opções, prestar muita atenção na separação das palavras, para que fique mais fácil a leitura.
        """,
        description="Agente que planeja viagens",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Cidade destino:{cidade_destino}\nEstado destino: {estado_destino}\nCidade de Origem: {cidade_origem}\nEstado Origem: {estado_origem}"
    # Executa o agente
    plano_de_viagem = call_agent(planejador, entrada_do_agente_planejador)
    return plano_de_viagem

######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar e-mails marketing da agência Lima's Tur.
            Você escreve e-mails marketing para uma empresa de viagens especializada em destinos nacionais.
            Utilize o destino fornecido no plano de destino e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de um e-mail cativante sobre o tema indicado.
            O e-mail deve ser engajador, informativo, e deve despertar no leitor uma vontade de conhecer a cidade,
            com linguagem simples e incluir uma chamada para ação no final.
            """,
        description="Agente redator de e-mails marketing engajadores"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho_gerado = call_agent(redator, entrada_do_agente_redator)
    return rascunho_gerado

##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em e-mails para marketing direto.
            Você tem um público diversificado, desde 18 anos até 70 anos. Use um tom de escrita adequado, envolvente, cativante,
            nem formal demais e nem coloquial demais.
            Revise o rascunho do e-mail abaixo sobre o destino indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado


data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
cidade_destino = input("❓ Por favor, informe a cidade de destino: ")
estado_destino = input("Informe o estado de destino: ")
cidade_origem = input("Informe a cidade de origem: ")
estado_origem = input("Informe o estado de origem: ")
destino = f"{cidade_destino}, {estado_destino} - {cidade_origem}, {estado_origem}"


########## Lógica do sistma de agentes ##########
if not destino:
    print("Você esqueceu de digitar o destino!")
else:
    print(f"Maravilha! Vamos então criar o seu roteiro de viagem para {cidade_destino.upper()} - {estado_destino.upper()}")

    destino_buscado = agente_buscador(destino, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(destino_buscado))
    print("--------------------------------------------------------------")

    plano_de_viagem = agente_planejador(cidade_destino, estado_destino, cidade_origem, estado_origem)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_viagem))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(destino, plano_de_viagem)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(destino, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, informe a cidade de destino: gramado
Informe o estado de destino: rs
Informe a cidade de origem: rio de janeiro
Informe o estado de origem: rj
Maravilha! Vamos então criar o seu roteiro de viagem para GRAMADO - RS

--- 📝 Resultado do Agente 1 (Buscador) ---



> Gramado, no Rio Grande do Sul, é um destino turístico popular com diversas atrações e atividades. Para planejar sua viagem em maio de 2025, aqui estão algumas sugestões e informações relevantes:
> 
> **Clima em Gramado em Maio:**
> 
> *   Em maio, Gramado tem temperaturas mais baixas, variando entre 8°C e 18°C.
> *   As manhãs e noites são geralmente mais frias, ideais para desfrutar de fondue e vinho.
> *   A previsão para 17 de maio de 2025 é de um dia ensolarado com temperatura mínima de 11.4°C e máxima de 26.3°C.
> 
> **Eventos em Maio de 2025:**
> 
> *   **Festa da Colônia:** Celebrada de 25 de abril a 12 de maio de 2025, no Expogramado, o evento destaca a cultura e a culinária dos descendentes de alemães, italianos e portugueses.
> *   **SICC 2025 - Salão Internacional do Couro e do Calçado:** De 13 a 15 de maio de 2025, no Serra Park.
> *   **Connection Terroirs do Brasil:** De 21 a 25 de maio de 2025, na Rua Coberta.
> *   **Tricofest:** De 9 de maio a 20 de junho, em Nova Petrópolis, com foco em moda de tricô.
> 
> **O que Fazer em Gramado:**
> 
> *   **Lago Negro:** Um dos pontos turísticos mais famosos, ideal para passeios de pedalinho.
> *   **Mini Mundo:** Parque com réplicas em miniatura de construções famosas.
> *   **Rua Coberta:** Rua charmosa com cobertura de vidro, lojas, cafés e restaurantes.
> *   **Snowland:** Parque de neve indoor com atividades como esqui e snowboard.
> *   **Museu de Cera Dreamland:**  Para tirar fotos com réplicas de celebridades.
> *   **Exceed Games Park:** Parque de diversões com diversas atrações para todas as idades.
> *   **Parque do Caracol (Canela):**  Divulgado a data de reabertura após melhorias.
> 
> **Restaurantes em Gramado:**
> 
> *   **Hard Rock Cafe Gramado:** Para fãs de rock'n'roll e hambúrgueres.
> *   **Edelweiss Restaurant:**  Especializado em fondue, com ambiente acolhedor.
> *   **Vue de la Vallée:**  Restaurante romântico com vista panorâmica e fondue.
> *   **DUO Cozinha Contemporânea:** Cozinha sofisticada com ingredientes frescos.
> *   **Olivas de Gramado:**  Restaurante em um parque que celebra a cultura do azeite.
> *   **Pizzaria Cara de Mau:**  Pizzaria temática com decoração de piratas e rodízio de pizzas.
> *    **Alice e o Chapeleiro:** Restaurante temático inspirado na história de Alice no País das Maravilhas.
> 
> **Hotéis em Gramado:**
> 
> *   **Hotel Ritta Höppner:** Considerado um dos melhores hotéis do mundo.
> *   **Hotel Laghetto Toscana:**  Oferece cancelamento grátis e café da manhã.
> *   **Hotel Monte Felice Bosque:**  Com café da manhã e desconto no Pix.
> *   **Wyndham Gramado Termas Resort:** Com desconto no Pix.
> *   **Buona Vitta Gramado Resort & Spa:**  Oferece café da manhã e desconto no Pix.
> *   **Hotel Laghetto Gramado:**  Com cancelamento grátis e café da manhã.
> 
> Lembre-se que em maio, Gramado comemora o Dia das Mães, então muitos restaurantes e hotéis podem oferecer programações temáticas.


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Olá! Com prazer, a Lima's Tur vai te ajudar a planejar sua viagem dos sonhos para Gramado, no Rio Grande do Sul, saindo do Rio de Janeiro. Preparei um plano de viagem detalhado com opções de transporte, acomodação, passeios e restaurantes, tudo em reais (R$) para facilitar o seu planejamento.
> 
> 
> ## Plano de Viagem Detalhado para Gramado (RS) saindo do Rio de Janeiro (RJ)
> 
> ### Transporte
> 
> **Opção 1: Aéreo (mais rápida)**
> 
> *   **Aeroporto de destino:** O aeroporto mais próximo de Gramado é o Aeroporto de Caxias do Sul (CXJ), mas o Aeroporto de Porto Alegre (POA) oferece mais voos e infraestrutura, ficando a cerca de 115 km de Gramado.
> *   **Companhias aéreas:** LATAM e GOL.
> *   **Preço médio da passagem:** R$ 611 a R$ 729 (ida e volta, com taxas).
> *   **Tempo de viagem:** Aproximadamente 3 horas (com escalas).
> *   **Observações:** Os voos podem ter escalas. Verifique a necessidade de bagagem despachada, pois algumas tarifas não incluem.
> *   **Do aeroporto a Gramado:**
>     *   **Transfer:** Agências de turismo oferecem traslados do aeroporto de Porto Alegre para Gramado.
>     *   **Ônibus:** A empresa Citral faz o trajeto de Porto Alegre a Gramado.
>     *   **Carro alugado:** Pode ser uma boa opção para ter mais flexibilidade na região.
> 
> **Opção 2: Ônibus (mais econômica, porém mais longa)**
> 
> *   **Empresas:** Útil, Viasul, Expresso Nossa Senhora da Penha.
> *   **Preço médio da passagem:** R$ 393,90 a R$ 1054,18.
> *   **Tempo de viagem:** 1 dia e 9 horas a 3 dias e 11 horas.
> *   **Observações:** Viagem longa com poltronas convencionais ou semi-leito.
> 
> **Recomendação:** Se o tempo for curto, a opção aérea é ideal. Caso prefira economizar e não se importe com a duração da viagem, o ônibus pode ser uma boa alternativa.
> 
> ### Hospedagem
> 
> Gramado oferece diversas opções de hotéis e pousadas para todos os bolsos.
> 
> **Hotéis no centro:**
> 
> *   **Hotel Colline de France:** Diárias a partir de R$ 1.000.
> *   **Hotel Laghetto Stilo Borges:** Diárias a partir de R$ 536.
> *   **Wish Serrano Resort:** Diárias a partir de R$ 596.
> *   **Hotel Fioreze Centro:** Diárias a partir de R$ 901.
> *   **Hotel Casa da Montanha:** Hotel romântico e com spa.
> 
> **Hotéis mais econômicos:**
> 
> *   **Hotel Kehl Haus:**.
> *   **Hotel Canto Verde:**.
> *   **Hotel Monte Felice Bosque:**.
> *   **Hotel Vovó Carolina:**.
> *   **Sky Hotel:**.
> 
> **Outras opções:**
> 
> *   **Pousadas:** La Vista, Pousada La Hacienda.
> *   **Chalés:** Hotel Fioreze Chalés, Villa Chalés Gramado.
> *   **Aluguel de temporada:** Apartamentos e casas.
> 
> **Recomendação:** Para quem busca praticidade, hospedar-se no centro é ideal, pois permite fácil acesso a restaurantes, lojas e atrações. Para economizar, procure hotéis e pousadas mais afastados ou alugue um apartamento.
> 
> ### Alimentação
> 
> Gramado é conhecida pela sua gastronomia diversificada, com opções para todos os gostos.
> 
> **Restaurantes:**
> 
> *   **Hard Rock Cafe Gramado:** Para fãs de rock e hambúrgueres.
> *   **Pizzaria Cara de Mau:** Pizzaria temática com rodízio e shows.
> *   **Edelweiss Restaurant:** Especializado em fondue.
> *   **Vue de la Vallée:** Ampla variedade de fondues.
> *   **Cucina Boniatto:** Comida italiana.
> *   **George III:** Pratos à la carte.
> *   **Italianinho Bella Cucina:** Culinária italiana com preço justo.
> *   **La Braise:** Carnes.
> *   **El Fuego e Malbec:** Sequências de fondue.
> 
> **Outras opções:**
> 
> *   **Cafés:** Rua Coberta.
> *   **Chocolaterias:** Prawer.
> *   **Self-service:** Vale Quanto Pesa.
> 
> **Recomendação:** Não deixe de experimentar o famoso fondue de Gramado! Para refeições mais em conta, procure restaurantes self-service ou peça pratos à la carte.
> 
> ### Passeios e Atrações Turísticas
> 
> Gramado oferece diversas atrações para todas as idades.
> 
> **Atrações imperdíveis:**
> 
> *   **Rua Coberta:** Charmosa rua com restaurantes, cafés e lojas.
> *   **Lago Negro:** Lago com pedalinhos e paisagens.
> *   **Mini Mundo:** Parque com réplicas de construções famosas.
> *   **Igreja de São Pedro:** Igreja com arquitetura em estilo romano.
> *   **Palácio dos Festivais:** Onde ocorre o Festival de Cinema de Gramado.
> *   **Snowland:** Parque de neve indoor.
> *   **Dreamland - Museu de Cera:** Esculturas de cera de celebridades.
> 
> **Outras atrações:**
> 
> *   **Mundo do Chocolate:**.
> *   **Acquamotion:**.
> *   **Le Jardin:**.
> *   **Praça das Etnias:**.
> *   **Rua Torta:**.
> *   **Fonte do Amor Eterno:**.
> *   **GramadoZoo:**.
> *   **Cristais de Gramado:**.
> *   **Rota Cervejeira:**.
> *   **Olivas de Gramado:**.
> 
> **Dicas:**
> 
> *   Aproveite os pontos turísticos gratuitos, como a Rua Coberta, a Igreja de São Pedro e a Fonte do Amor Eterno.
> *   Considere adquirir ingressos antecipados para os parques e museus para evitar filas.
> *   Visite também a cidade vizinha de Canela, que oferece atrações como o Parque do Caracol e a Catedral de Pedra.
> 
> ### Custo de Vida
> 
> O custo de vida em Gramado pode ser comparado ao de grandes centros como Rio de Janeiro e São Paulo.
> 
> *   **Aluguel:** Apartamentos de 2 quartos no centro variam de R$ 2.500 a R$ 5.000.
> *   **Alimentação:** Os preços dos restaurantes turísticos podem ser elevados, mas há opções mais acessíveis.
> *   **Transporte:** Caminhar pelo centro é fácil, mas para outras atrações, considere alugar um carro ou usar transporte por aplicativo.
> 
> **Observação:** O custo de vida pode ser 27% mais barato em Gramado do que no Rio de Janeiro.
> 
> ### Observações Finais
> 
> *   **Melhor época para visitar:** Gramado é um destino encantador em todas as estações. O Natal Luz é um evento especial, mas a cidade também é linda no inverno e na primavera.
> *   **Segurança:** Gramado é considerada uma cidade segura, permitindo que os visitantes caminhem tranquilamente pelas ruas.
> *   **Clima:** Prepare-se para o clima frio, principalmente no inverno.
> 
> Espero que este plano de viagem detalhado te ajude a organizar sua viagem dos sonhos para Gramado! A Lima's Tur está à disposição para auxiliar em todos os detalhes.


--------------------------------------------------------------
